### Import Libraries


In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
import joblib

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

### Import Data

In [2]:
train = pd.read_csv('../data/readmissions_train.csv')
train.drop(['diag_1_desc','diag_2_desc','diag_3_desc'],axis=1,inplace=True)

X = train.drop('readmitted',axis=1)
y = train.pop('readmitted')

### Function for Keras Model

In [3]:
#First I define my keras model

def create_model():
    optimizer='adagrad'
    kernel_initializer='glorot_uniform'
    dropout=0.2
    
    model = Sequential()
    model.add(Dense(64,activation='relu',kernel_initializer=kernel_initializer))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer))

    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])

    return model

### Define Preprocessing step per type of column

In [4]:

#Preprocessing for numerical features
numeric_features = list(set(X.columns) - set(list(X.select_dtypes('object').columns)))
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

#Preprocessing for categorical features
categorical_features = list(train.select_dtypes('object').columns)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

#Preprocessor with all of the steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

### Fit the Preprocessing Pipeline

In [5]:
# Full preprocessing pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

#Train the model-Pipeline
pipeline.fit(X,y)

#Preprocess x
preprocessed = pipeline.transform(X)

### Train the Keras Classifier

In [6]:
#Create the keras model
clf = KerasClassifier(build_fn=create_model, verbose=0)
clf.fit(preprocessed,y)

### Save Custom Model files

In [7]:
joblib.dump(pipeline,'custom_model/preprocessing.pkl')
clf.model.save('custom_model/model.h5')

### Verify custom model integrity with DRUM

In [ ]:
!drum validation --code-dir ./custom_model --input ../data/readmissions_test.csv --target-type binary --positive-class-label yes --negative-class-label no

2020-11-30 00:34:37.577761: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:34:37.595374: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fad85b49740 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:34:37.595426: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:34:38,617 WARNING tensorflow:  From

2020-11-30 00:35:08.509357: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:35:08.537098: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8ba04f9f50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:35:08.537145: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:35:09,929 WARNING tensorflow:  From

2020-11-30 00:35:40.260874: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:35:40.276759: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe5b86972b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:35:40.276804: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:35:41,271 WARNING tensorflow:  From

2020-11-30 00:36:09.546875: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:36:09.563256: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8d586f24d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:36:09.563290: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:36:10,465 WARNING tensorflow:  From

2020-11-30 00:36:36.050163: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:36:36.063598: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd08440fe40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:36:36.063632: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:36:36,882 WARNING tensorflow:  From

2020-11-30 00:37:04.308692: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:37:04.324500: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f98397b0520 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:37:04.324558: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:37:05,663 WARNING tensorflow:  From